In [53]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/16.NiftyOptions/')

df_nifty=pd.read_csv('01.RawData/Nifty2017.csv',dtype=str)
data_ce=pd.read_csv('01.RawData/OPTIDX_NIFTY_CE_09-12-2016_TO_08-12-2017.csv',dtype=str)
data_pe=pd.read_csv('01.RawData/OPTIDX_NIFTY_PE_09-12-2016_TO_08-12-2017.csv',dtype=str)
df_options=pd.concat([data_ce,data_pe])

In [129]:
df_nifty['Close']=df_nifty['Close'].astype(np.float64)
df_nifty['Date']=pd.to_datetime(df_nifty['Date'], format='%d-%b-%Y')
df_options['Close']=df_options['Close'].astype(np.float64)
df_options['Date']=pd.to_datetime(df_options['Date'], format='%d-%b-%Y')
df_options['Expiry']=pd.to_datetime(df_options['Expiry'], format='%d-%b-%Y')

In [57]:
df_nifty['PrevClose']=np.roll(df_nifty['Close'],1)
df_nifty['PrevClose'][0]=np.nan
df_nifty['PctChange']=100*(df_nifty['Close']/df_nifty['PrevClose']-1)
df_nifty.head()


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),PrevClose,PctChange
0,2016-12-12,8230.65,8230.65,8154.45,8170.80,111432551,5026.02,NaN,NaN
1,2016-12-13,8196.15,8228.85,8155.80,8221.80,189739559,8976.54,8170.80,0.624174
2,2016-12-14,8229.35,8229.40,8165.10,8182.45,149132520,6763.63,8221.80,-0.478606
3,2016-12-15,8128.40,8225.90,8121.95,8153.60,187425988,8579.44,8182.45,-0.352584
4,2016-12-16,8178.20,8178.70,8127.45,8139.45,209268331,9854.07,8153.60,-0.173543


In [84]:
bins=pd.qcut(df_nifty['PctChange'],10,retbins=True)[1]
bins[0]=-100
bins[-1]=100
bins=[-100,-0.6,-0.3,-0.15,0.15,0.3,0.6,100]
bin_labels=['%d.(%0.2f,%0.2f)'%(i,x,y) for i,(x,y) in enumerate(zip(bins[:-1],bins[1:]))]
df_nifty['PctChangeBin']=pd.cut(df_nifty['PctChange'],bins=bins,labels=bin_labels)
df_nifty['PctChangeBin'].value_counts().sort_index()

0.(-100.00,-0.60)    31
1.(-0.60,-0.30)      22
2.(-0.30,-0.15)      22
3.(-0.15,0.15)       70
4.(0.15,0.30)        22
5.(0.30,0.60)        30
6.(0.60,100.00)      51
Name: PctChangeBin, dtype: int64

In [211]:
#for i,row in df_nifty.head().iterrows():
def get_best_trade(row,profit):
    opt_data=df_options[df_options['Date']==row['Date'].date()]
    opt_data['Days_to_Expiry']=(opt_data['Expiry']-opt_data['Date']).dt.days
    opt_data=opt_data[opt_data['Days_to_Expiry']>15]
    opt_data=opt_data[opt_data['Days_to_Expiry']==opt_data['Days_to_Expiry'].min()]
    opt_data=opt_data.groupby(['Expiry','Strike Price','Option Type'])['Close'].max().unstack()
    opt_data['Diff']=np.abs(opt_data['CE']-opt_data['PE'])
    opt_data=opt_data[opt_data['Diff']==opt_data['Diff'].min()].reset_index().ix[0]
    output=opt_data.copy()
    output['TargetProfit']=profit
    output['NiftyClose']=row['Close']
    output['Date']=row['Date']
    output['CP']=output['CE']+output['PE']
    opt_data=df_options[np.logical_and(df_options['Expiry']==opt_data['Expiry'],df_options['Strike Price']==opt_data['Strike Price'])]
    opt_data=opt_data[opt_data['Date']>row['Date'].date()]
    opt_data=opt_data.groupby(['Date','Option Type'])['Close'].max().unstack()
    if len(opt_data)==0 or 'CE' not in opt_data.columns or 'PE' not in opt_data.columns :
        return pd.Series(output)
    opt_data['SP']=opt_data['CE']+opt_data['PE']
    opt_data['Date']=opt_data.index
    if opt_data['SP'].max()<(1+profit/100.0)*output['CP']:
        return pd.Series(output)
    opt_data=opt_data[opt_data['SP']>(1+profit/100.0)*output['CP']].ix[0]
    output['SP_CE']=opt_data['CE']
    output['SP_PE']=opt_data['PE']
    output['SP']=opt_data['SP']
    output['SaleDate']=opt_data['Date']
    output['Days']=opt_data['Date']-row['Date']
    output['Profit']=100*(output['SP']/output['CP']-1)
    return pd.Series(output)

def get_profit_attributes(profit):
    output_df=df_nifty.apply(get_best_trade,axis=1,profit=profit)
    try:
        output_df['Date']=pd.to_datetime(output_df['Date'])
        output_df['SaleDate']=pd.to_datetime(output_df['SaleDate'])
        output_df['Days']=pd.to_timedelta(output_df['Days']).dt.days
        output_df=output_df[['Date','NiftyClose','Expiry','Strike Price','CE','PE','CP','TargetProfit','SaleDate','Days','SP_CE','SP_PE','SP','Profit']]
    except:
        pass
    return output_df

profits=pd.concat([get_profit_attributes(profit) for profit in [5,10,20,25,30]])
#profits=pd.concat([get_profit_attributes(profit) for profit in [5]])

C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://p

In [210]:
profits.head()

Option Type,Date,NiftyClose,Expiry,Strike Price,CE,PE,CP,TargetProfit,SaleDate,Days,SP_CE,SP_PE,SP,Profit
0,2016-12-12,8170.80,2016-12-29,8200,91.00,111.30,202.30,5,2016-12-26,14.0,1.30,291.20,292.50,44.587247
1,2016-12-13,8221.80,2016-12-29,8250,83.45,107.50,190.95,5,2016-12-22,9.0,2.60,252.30,254.90,33.490443
2,2016-12-14,8182.45,2017-01-25,8200,178.15,149.70,327.85,5,2017-01-25,42.0,395.25,0.05,395.30,20.573433
3,2016-12-15,8153.60,2017-01-25,8200,155.60,150.80,306.40,5,2016-12-26,11.0,39.90,293.95,333.85,8.958877
4,2016-12-16,8139.45,2017-01-25,8200,138.20,156.85,295.05,5,2016-12-26,10.0,39.90,293.95,333.85,13.150314


In [212]:
profit_pct=profits.groupby(profits['TargetProfit'])[['TargetProfit','Profit']].count()
profit_pct['Pct']=profit_pct['Profit']/profit_pct['TargetProfit']
profit_pct

,TargetProfit,Profit,Pct
TargetProfit,,,
5,249,185,0.742972
10,249,161,0.646586
20,249,130,0.522088
25,249,111,0.445783
30,249,100,0.401606


In [183]:
profits.groupby(profits['TargetProfit'])[['Days']].apply(lambda x: x.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.90,0.95,0.99]).T)


,,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
TargetProfit,,,,,,,,,,,,,,,
5,Days,185.0,8.837838,6.786790,1.0,1.00,1.00,2.0,4.00,7.0,13.00,18.0,21.00,28.80,42.0
10,Days,161.0,11.813665,8.198479,1.0,1.60,2.00,3.0,6.00,10.0,16.00,22.0,27.00,41.40,42.0
20,Days,130.0,14.623077,9.296382,2.0,2.00,3.00,4.9,7.25,13.0,20.00,27.0,35.55,41.71,42.0
25,Days,111.0,15.558559,9.509502,2.0,2.10,4.00,5.0,8.00,14.0,20.00,27.0,36.00,40.90,42.0
30,Days,100.0,15.920000,9.235067,3.0,3.00,4.00,6.0,9.00,14.0,21.00,30.2,36.00,37.03,40.0
40,Days,88.0,17.840909,10.425308,3.0,3.00,6.00,7.0,9.00,14.5,25.25,34.3,37.00,40.00,40.0
50,Days,74.0,18.851351,10.962804,4.0,5.46,6.65,7.0,9.00,15.0,28.75,34.7,38.70,41.00,41.0
75,Days,44.0,19.204545,9.610023,4.0,4.86,7.00,8.3,13.00,17.0,27.25,33.0,34.85,41.57,42.0
100,Days,20.0,23.900000,4.897905,15.0,15.38,16.90,17.9,20.75,23.0,28.00,29.1,30.15,32.43,33.0


In [196]:
porifts1=profits.dropna()
pd.crosstab(np.floor(porifts1.Date.dt.day/5),porifts1['TargetProfit'])

TargetProfit,5,10,20,25,30,40,50,75,100
Date,,,,,,,,,
0.0,21,19,16,14,14,13,12,7,4
1.0,32,27,20,19,16,14,12,11,4
2.0,27,22,20,18,15,13,11,8,1
3.0,31,28,20,17,15,13,10,1,0
4.0,34,32,27,21,19,17,13,6,1
5.0,31,26,21,19,18,15,13,10,9
6.0,9,7,6,3,3,3,3,1,1


In [221]:
profits['days_to_expiry']=(profits['Expiry']-profits['Date']).dt.days
profits['days_to_expiry_bin']=pd.cut(profits['days_to_expiry'],bins=[0,16,28,38,90])
profit_pct=profits.groupby(['days_to_expiry_bin','TargetProfit'])[['TargetProfit','Profit']].count()
profit_pct['Pct']=profit_pct['Profit']/profit_pct['TargetProfit']
profit_pct.unstack().to_clipboard()

In [223]:
profits1=profits[(profits['days_to_expiry']>=28)&(profits['days_to_expiry']<=38)]
profits1.groupby(profits1['TargetProfit'])[['Days']].apply(lambda x: x.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.90,0.95,0.99]).T)


,,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
TargetProfit,,,,,,,,,,,,,,,
5,Days,83.0,7.783133,6.219655,1.0,1.00,1.0,1.2,3.0,6.0,11.00,16.0,18.9,25.54,28.0
10,Days,75.0,10.933333,7.228572,1.0,1.00,2.0,3.0,4.5,8.0,16.00,21.0,23.5,28.26,29.0
20,Days,64.0,14.218750,8.895279,2.0,2.00,4.0,4.3,7.0,13.5,19.25,26.7,34.1,36.37,37.0
25,Days,55.0,16.290909,9.186691,2.0,3.08,4.7,5.4,9.0,14.0,20.50,27.8,36.0,37.00,37.0
30,Days,52.0,17.807692,9.707564,4.0,4.00,5.0,7.0,11.0,15.0,23.50,33.9,36.0,37.00,37.0
